In [ ]:
"""
approach is keyword matching match lyrics of new songs with songs listened by user 
to do that we need to follow 2 steps 
1. extract basic or signature lyrics of song ( we use TfidVectorizer from sklearn)
2. match those lyrics with new songs and recommend if they are similar (we use cosine_similairty from sklearn)
"""
"""
Term Frequency-Inverse Document Frequency (TF-IDF)
is method we follow which extract information from sentence here it is song
"""

In [1]:
import numpy as np
import pandas as pd

In [2]:
from typing import List, Dict

In [3]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [4]:
songs = pd.read_csv('songdata.csv')

In [5]:
songs.head()

,artist,song,link,text
0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look at her face, it's a wonderful face \nAnd..."
1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,"Take it easy with me, please \nTouch me gentl..."
2,ABBA,As Good As New,/a/abba/as+good+as+new_20003033.html,I'll never know why I had to go \nWhy I had t...
3,ABBA,Bang,/a/abba/bang_20598415.html,Making somebody happy is a question of give an...
4,ABBA,Bang-A-Boomerang,/a/abba/bang+a+boomerang_20002668.html,Making somebody happy is a question of give an...


Because of the dataset being so big, we are going to resample only 5000 random songs.

In [6]:
songs = songs.sample(n=5000).drop('link', axis=1).reset_index(drop=True)

We can notice also the presence of `\n` in the text, so we are going to remove it.

In [7]:
songs['text'] = songs['text'].str.replace(r'\n', '')

In [8]:
tfidf = TfidfVectorizer(analyzer='word', stop_words='english')

In [9]:
lyrics_matrix = tfidf.fit_transform(songs['text'])

In [10]:
cosine_similarities = cosine_similarity(lyrics_matrix) 

In [11]:
similarities = {}

In [12]:
for i in range(len(cosine_similarities)):
    similar_indices = cosine_similarities[i].argsort()[:-50:-1]     
    similarities[songs['song'].iloc[i]] = [(cosine_similarities[i][x], songs['song'][x], songs['artist'][x]) for x in similar_indices][1:]

In [13]:
class ContentBasedRecommender:
    def __init__(self, matrix):
        self.matrix_similar = matrix

    def _print_message(self, song, recom_song):
        rec_items = len(recom_song)
        
        print(f'The {rec_items} recommended songs for {song} are:')
        for i in range(rec_items):
            print(f"Number {i+1}:")
            print(f"{recom_song[i][1]} by {recom_song[i][2]} with {round(recom_song[i][0], 3)} similarity score") 
            print("--------------------")
        
    def recommend(self, recommendation):
        song = recommendation['song']
        number_songs = recommendation['number_songs']
        recom_song = self.matrix_similar[song][:number_songs]
        self._print_message(song=song, recom_song=recom_song)

In [14]:
recommedations = ContentBasedRecommender(similarities)

In [15]:
recommendation = {
    "song": songs['song'].iloc[10],
    "number_songs": 4 
}

In [16]:
recommedations.recommend(recommendation)

The 4 recommended songs for Buck Dich (English) are:
Number 1:
25 by Veruca Salt with 0.403 similarity score
--------------------
Number 2:
Lovely To See You by Moody Blues with 0.293 similarity score
--------------------
Number 3:
Is It Me Is It You by Lenny Kravitz with 0.187 similarity score
--------------------
Number 4:
Already There by Verve with 0.162 similarity score
--------------------


In [17]:
recommendation2 = {
    "song": songs['song'].iloc[120],
    "number_songs": 4 
}

In [18]:
recommedations.recommend(recommendation2)

The 4 recommended songs for Man Smart, Woman Smarter are:
Number 1:
Hold What You've Got by Hank Williams Jr. with 0.22 similarity score
--------------------
Number 2:
Wild Women Do by Natalie Cole with 0.216 similarity score
--------------------
Number 3:
Honky Tonk Songs by Dolly Parton with 0.214 similarity score
--------------------
Number 4:
Saw Mill (set I) by Grateful Dead with 0.143 similarity score
--------------------
